In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home/")
data = loader.load()
data

In [ ]:
type(data[0])

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

In [ ]:
from langchain_ollama import OllamaEmbeddings
embedding = OllamaEmbeddings(model="gemma:2b")

In [ ]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(texts, embedding)

In [ ]:
vectorstore.similarity_search("What is LangSmith?")

## RETRIEVAL CHAIN & DOCUMENT CHAIN

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [23]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the context below.
    {context}
    """
)

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt=prompt)

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"langsmith",
    "context": [Document(page_content="LangSmith is a comprehensive platform designed to help developers build, evaluate, and monitor language models. It offers features such as experiment tracking, model evaluation, and deployment tools to streamline the development process. With LangSmith, users can easily manage their language model projects and collaborate with team members.")]
})

In [ ]:
retriever = vectorstore.as_retriever()

chain = retriever | document_chain
chain

In [ ]:
from langchain_classic.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)
retrieval_chain

In [ ]:
retrieval_chain.invoke({"input":"What is LangSmith?"})